<img src="../../contrib/img/logo.png" width=150 align="left" /> <img src="../../contrib/img/logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>MAPS</font></center>

> # <font color='steelblue'> <b>ROUTES ON MAP</font>

### <font color='steelblue'>Description</font>

 Dibujar un conjunto de 1 a n rutas de diferente longitud cada una sobre un mismo mapa.

INFO DE PARTIDA
- Partimos de una lista a modo de conjunto de puntos en formato (Lat, Long) para poder calcular cada ruta.
- Cada una de las listas tiene la forma [48, 62, 36, 78, 7, 49, 6, 39, 69, 64, 8, 47]. Los números se corresponden con el índice de un fichero que tiene las columnas Lat y Long de cada punto de la ruta. 
- La extracción de los puntos para generar las ruta a partir del fichero viene ya dada en el código. 
- Esto NO HAY QUE HACERLO YA VIENE HECHO.

RUTAS
- Con esa lista de puntos lo pirmero que se debe hacer es obtener las rutas. Para ello, usaremos Open Street Maps. Posteriormente se ampliará para obtenerlas a partir de Google.
- Se debe poder parametrizar para las rutas lo que permita OSM en ese sentido (e.g. cars, ...)
- Se debe extraer toda la información posible de la ruta que de OSM: distancia, tiempo, ...
- Toda la información que se obtenga de la rutas de OMS (distancia, tiempo, ...) se debe guardar en un dataframe para su uso posterior.

MAPA
- A partir de las turtas calculadas ya se procede a realizar el mapa.
- Cada ruta debe diferenciarse de las demás por un color.
- En cada ruta debe diferenciarse claramente el punto de partida (primer punto) y el de llegada (último punto).
- Llevará el mapa un cuadro de selección en la esquina superior izquierda de las rutas que se desean seleccionar para que se muestren en el mapa
- Llevará el mapa una leyenda en la esquina inferior derecha con la información de la/s ruta/s seleccionada/s (num de puntos, distancia, ...) o el total de todas si no se marca ninguna



<img src="../../contrib/img/routes.jpg" width=550 align="left" />

## <font color='green'>Settings</font>

Librerias a instalar:
 - dash
 - geopandas
 - numpy
 - openrouteservice
 - pandas
 - plotly
 - dash
 - jupyter_dash (si se despliega desde notebook solo)

In [1]:
# Libraries to use
import os
from datetime import date
from itertools import chain

import geopandas as gpd
import numpy as np
import openrouteservice
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html
from dash import dcc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash
from openrouteservice import convert
from openrouteservice.directions import directions

## <font color='green'>Data Load</font>

<font color='green'><b> PEDIDOS</b></font>

In [2]:
# NOTA: Cambiar por la ruta relativa a los datos
PEDIDOS = pd.read_csv("../../contrib/data/PEDIDOS.csv")

In [3]:
PEDIDOS.head()

,Id_Pedido,Fecha,Lat,Long,Direccion,CP,Id_Paquete,Urgente,Ventana_Inicio,Ventana_Fin,Devolucion,Id_Paquete_Devol,Id_Cliente
0,1,17/11/2022,40.404894,-3.683088,"Avenida Ciudad de Barcelona N 108 Esc.1 1A, Ma...",28007,111,NO,0:00:00,23:59:59,SI,88.0,1
1,2,17/11/2022,40.416078,-3.702817,"Calle de Cadiz N 1 1 1, Madrid",28012,17,NO,0:00:00,23:59:59,NO,NaN,2
2,3,17/11/2022,40.414122,-3.704077,"Calle de Concepcion jeronima N 5 1 DR, Madrid",28012,4,SI,13:27:27,15:40:34,SI,81.0,3
3,4,17/11/2022,40.410154,-3.695928,"Calle del Marques de Toca N 12 Esc.1 AT 1, Madrid",28012,115,NO,0:00:00,23:59:59,NO,NaN,4
4,5,17/11/2022,40.410179,-3.697674,"Calle del salitre N 8 3 IZ, Madrid",28012,58,NO,12:00:39,16:07:01,NO,NaN,5


## <font color='green'>ROUTE EXTRACTION</font>

Se calculan a continuación rutas a partir de un fichero para que sirvan de ejemplo.

Se debe trabajar para el mapa con la lista final de rutas generada.

Se generan en dos formatos para elegir el más óptimo para hacer el mapa.

In [4]:
# String de la columna que contiene las latitudes
latStr = 'Lat'
# String de la columna que contiene las longitudes
lonStr = 'Long'

# Pasamos el dataframe a geodataframe
gdf = gpd.GeoDataFrame(PEDIDOS, geometry=gpd.points_from_xy(PEDIDOS[latStr], PEDIDOS[lonStr]))

# Creamos las diferentes rutas en formato [0, 1, 2, 3, 4] donde cada numero es el indice de un punto en el fichero
rutas = []
# Añadimos las rutas deseadas
rutas.append([1,5,3,2,6])
rutas.append([50,65,23,100])
rutas.append([31,25,26,27,28,29,30,31,65,90,34,66])

# Recorremos las diferentes rutas a pintar
rutas_puntos1=[]
rutas_puntos2=[]
for i, r in enumerate(rutas):
    ruta = ''
    for i in range(len(gdf)):
        ruta += f'{gdf.geometry.x.tolist()[i]},{gdf.geometry.y.tolist()[i]};' 
    ruta = ruta[:-1]
    puntosRecursos = gdf.iloc[r][[latStr, lonStr]].values.tolist()
    rutas_puntos1.append([ruta]) 
    rutas_puntos2.append(puntosRecursos) 

Rutas para trabajar

In [5]:
RUTAS_MAP=rutas_puntos2

## <font color='green'>PARAMETERS</font>

Variables

In [6]:
# Titulo del mapa
titulo = 'Rutas'
# Tipo de mapa
estilo_mapa='open-street-map' # "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain",                           
            # "stamen-toner" or "stamen-watercolor" 
# Tipo de medio de transporte
driving_profile = "driving-car" #"driving-car", "driving-hgv", "foot-walking", "foot-hiking", "cycling-regular", "cycling-road","cycling-mountain",
    # "cycling-electric"
    
# API KEY openrouteservice (OSM)
API_KEY = "5b3ce3597851110001cf6248e377711d217e471391ee7c5e0969bcd3"


# Paleta de colores
color_palette = ['black', 'olive','lime','aqua', 'teal','fuchsia',
         'red','#b03a2e','#cb4335','#e74c3c','#ec7063','#f1948a','#f5b7b1',
         'green','#196f3d','#229954','#27ae60','#52be80','#7dcea0','#a9dfbf',
         'maroon','#935116','#ca6f1e','#e67e22','#eb984e','#f0b27a','#f5cba7',
         'gray','#797d7f','#a6acaf','#bdc3c7','#cacfd2','#d7dbdd','#e5e7e9',
         'purple','#76448a','#884ea0','#9b59b6','#af7ac5','#c39bd3','#d7bde2',
         'blue','#1f618d','#2471a3','#2980b9','#5499c7','#7fb3d5','#a9cce3',
         'yellow','#d4ac0d','#f1c40f','#f4d03f','#f7dc6f','#f9e79f','#fcf3cf',]

# Zoom del mapa
zoom = 10

## <font color='green'>OSM ROUTES GENERATION</font>

Generamos las rutas a partir de OMS y con toda la info que se extraiga de ellas se guara en un dataframe y fichero

In [7]:
client = openrouteservice.Client(key=API_KEY) # Specify your personal API key

In [8]:
def swap_latlong(coords):
    p1, p2 = zip(*coords)
    return list(zip(p2, p1))


def get_route_between(client, coords):
    longlat_coords = swap_latlong(coords)
    partial_paths = []
    distance = 0
    duration = 0
    for prev_coords, next_coords in zip(longlat_coords[0:], longlat_coords[1:]):
        partial_best_route = directions(client, (prev_coords, next_coords), profile=driving_profile)["routes"][0]
        distance += partial_best_route["summary"]["distance"]
        duration += partial_best_route["summary"]["duration"]
        partial_path = convert.decode_polyline(partial_best_route["geometry"])["coordinates"]
        partial_paths.append(swap_latlong(partial_path))
    
    return {"coords": coords, "distance": distance, "duration": duration, "partial_paths": partial_paths}

In [9]:
keep_atts = ["Id_Paquete", "Urgente", "Ventana_Inicio", "Ventana_Fin", "Devolucion", "Id_Cliente", "Id_Paquete_Devol", "CP"]

OSM_df_data = []

for order_idxs in rutas:
    try:
        order_points_df = PEDIDOS[PEDIDOS["Id_Pedido"].isin(order_idxs)]
        coords = list(zip(order_points_df["Lat"], order_points_df["Long"]))
        route_plan = get_route_between(client, coords)
        
        order_points_df_sorted = order_points_df
        
        order_with_plan = {
            **route_plan,
            **{k: order_points_df_sorted[k].tolist() for k in keep_atts}
        }
        
        OSM_df_data.append(order_with_plan)
    except Exception as e:
        print(e)
        
OSM_df = pd.DataFrame(OSM_df_data)
OSM_df["partial_paths_join"] = OSM_df["partial_paths"].apply(lambda x: list(chain(*x)))

In [10]:
RUTAS_OMS = OSM_df

In [11]:
RUTAS_OMS.to_csv("RUTAS_OMS.csv")

## <font color='green'>GOOGLE ROUTES GENERATION</font>

Generamos las rutas a partir de GOOGLE y con toda la info que se extraiga de ellas se guara en un dataframe y fichero.
QUEDA PENDIENTE HASTA EU TENGAMOS ACCESO A GOOGLE MAPS

In [12]:
# 


## <font color='green'>ROUTE MAP</font>

**Seleccionamos un dataframe de datos y lo asignamos a RUTAS_MAPA**. Por ahora solo esta disponible el dataframe de OSM. Las visualizaciones se harán con este dataset.

In [13]:
RUTAS_MAPA = OSM_df

Visualizamos las rutas obtenidas y seleccionadas (OMS o Google) en el mapa

Esta celda crea la visualización únicamente, recibiendo un dataframe de rutas. Dicha visualización se devuelve como resultado de la función, por lo que no está acoplada al notebook y puede ser utilizada por "código" externo.

In [14]:
def plot_paths(paths): 
    traces = []
    
    min_lat, max_lat = float("+inf"), float("-inf")
    min_long, max_long = float("+inf"), float("-inf")
    
    for i, path_info in enumerate(paths.to_dict(orient="records")):
        lats, longs = zip(*path_info["partial_paths_join"])
        distance = path_info["distance"]
        duration = path_info["duration"]
        lats_checkpoints, longs_checkpoints = zip(*path_info["coords"])
        id_route = f"Ruta {i}"
        route_color = color_palette[i]
        
        min_lat = min(min_lat, min(lats_checkpoints))
        max_lat = max(max_lat, max(lats_checkpoints))
        min_long = min(min_long, min(longs_checkpoints))
        max_long = max(max_long, max(longs_checkpoints))

        traces.append(go.Scattermapbox(
            name=id_route,
            lat=lats,
            lon=longs,
            legendgroup=id_route,
            marker={"color": route_color},
            mode='lines'))
        
        hover_texts = [
            "<br>".join(f"<b>{k}</b>: {path_info[k][i]}" for k in keep_atts) for i in range(len(lats_checkpoints))
        ]
        traces.append(go.Scattermapbox(
            name=f"Ruta {i}",
            lat=lats_checkpoints,
            lon=longs_checkpoints,
            mode="markers",
            hovertemplate="%{text}",
            showlegend = False,
            text=hover_texts,
            legendgroup=id_route,
            marker={"size": 12, "color": route_color}
        ))
    
    fig = go.Figure(data=traces)
    fig.update_layout(title_text = titulo)
    fig.update_layout(mapbox = {
        'style': estilo_mapa,
        'center': {
            'lat': (max_lat + min_lat) / 2,
            'lon': (max_long + min_long) / 2,
        },
        'zoom': zoom}
    )
    fig.update_layout(legend_title_text="Rutas:")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})    
    return fig

Función que calcula las estadísticas a mostrar de un conjunto de rutas. Si añadimos nuevas estadísticas aquí se verán automáticamente reflejadas en la gráfica.

In [15]:
def calculate_statistics_text(routes_df, include_ids):
    routes_df = routes_df.iloc[include_ids]
    
    number_points = routes_df["coords"].apply(len).sum()
    distance = routes_df["distance"].sum()
    duration = routes_df["duration"].sum()
    
    route_info = f"""<b>Información de las rutas:</b> <br>
    - Número de puntos {number_points} <br>
    - Distancia total {round(distance / 1000, 2)}km <br>
    - Tiempo total {round(duration / 60, 2)}min <br>
    <br>
    <br>
<b>Selector de rutas:</b> <br>
    """
    return route_info

Creamos la aplicación de Dash con las visualizaciones. Si se ejecuta fuera de jupyter cambiar app=JupyterDash() por app=dash.Dash(). He juntado las dos leyendas en una por tema de espacios,
después de varias pruebas creo que se aprovecha mejor el espacio y ancho. Además así no se solapa información con el mapa.

Si se pincha sobre el inicio o final de una ruta se muestran los detalles asociados en el Dataframe. Se podría poner sobre el trazo de la ruta si fuese necesario.

In [16]:
app = JupyterDash()
app.layout = html.Div([
    dcc.Store(id='selected-memory'),
    dcc.Graph(
        id='route-map',
        config={'editable':True},
        figure=plot_paths(RUTAS_MAPA)
    ),
])

figureg = None

@app.callback(
    [Output('route-map', 'figure'), Output('selected-memory', 'data')],
    [Input('route-map', 'figure'), Input('route-map', 'restyleData'), State('selected-memory', 'data')])
def update_statistics_legend(figure, restyleData, data):
    if data is None:
        data = dict()
        
    if restyleData is not None:
        disable_event = restyleData[0]["visible"][0] == "legendonly"
        trace_id = int(restyleData[1][0] // 2)
    
        if disable_event:
            data[trace_id] = True
        elif trace_id in data:
            del data[trace_id]
        
    available_ids = RUTAS_MAPA.index.tolist()
    available_ids = list(set(available_ids) - set(data.keys()))
    figure["layout"]["legend"]["title"]["text"] = calculate_statistics_text(RUTAS_MAPA, available_ids)
    return figure, data

app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


## <font color='green'>ROUTE VALUES</font>

In [17]:
# 
RUTAS_DF = RUTAS_OMS

Dataframe de valores ontenidos de las rutas (puntos, tiempo, distancia, etc). Cada fila es una ruta. Las coordenadas parciales (partial_paths) es una lista de listas. La lista i indica las coordenadas desde el checkpoint i (coords[i]) hasta el checkpoint i+1 (coords[i+1]). La distancia y duracion es el acumulado de las rutas parciales. El resto de atributos es la concatenación de las rutas parciales. Por ejemplo, CP[i] es la información de la ruta que empieza en coords[i]).

In [18]:
RUTAS_DF

,coords,distance,duration,partial_paths,Id_Paquete,Urgente,Ventana_Inicio,Ventana_Fin,Devolucion,Id_Cliente,Id_Paquete_Devol,CP,partial_paths_join
0,"[(40.404893738, -3.6830877896), (40.4160778229...",6422.1,1574.1,"[[(40.40496, -3.68305), (40.40495, -3.683), (4...","[111, 17, 4, 58, 19]","[NO, NO, SI, NO, NO]","[0:00:00, 0:00:00, 13:27:27, 12:00:39, 0:00:00]","[23:59:59, 23:59:59, 15:40:34, 16:07:01, 23:59...","[SI, NO, SI, NO, SI]","[1, 2, 3, 5, 6]","[88.0, nan, 81.0, nan, 90.0]","[28007, 28012, 28012, 28012, 28012]","[(40.40496, -3.68305), (40.40495, -3.683), (40..."
1,"[(40.4041819275, -3.6931606697), (40.417945476...",10484.1,1696.1,"[[(40.40412, -3.69317), (40.40407, -3.69221), ...","[176, 35, 162, 124]","[NO, NO, NO, NO]","[5:52:21, 0:00:00, 0:00:00, 0:00:00]","[10:10:49, 23:59:59, 23:59:59, 23:59:59]","[NO, NO, NO, NO]","[23, 50, 65, 100]","[nan, nan, nan, nan]","[28045, 28013, 28007, 28009]","[(40.40412, -3.69317), (40.40407, -3.69221), (..."
2,"[(40.4142261891, -3.7025971319), (40.418553942...",34739.2,5025.1,"[[(40.41391, -3.70279), (40.41393, -3.70285), ...","[144, 183, 143, 107, 89, 87, 172, 98, 162, 46,...","[NO, NO, NO, NO, NO, NO, NO, SI, NO, NO, NO]","[0:00:00, 16:23:16, 0:00:00, 0:00:00, 0:00:00,...","[23:59:59, 20:34:30, 23:59:59, 23:59:59, 23:59...","[NO, NO, SI, NO, NO, NO, NO, NO, NO, NO, NO]","[25, 26, 27, 28, 29, 30, 31, 34, 65, 66, 90]","[nan, nan, 94.0, nan, nan, nan, nan, nan, nan,...","[28012, 28014, 28007, 28005, 28014, 28004, 280...","[(40.41391, -3.70279), (40.41393, -3.70285), (..."
